# Solving Eigenproblem

In this notebook we will get the Hessian and solve the first 64 modes of it using Inching-JDMHD, by default the free modes are excluded from considerations.
Note that the first-ever run of Inching on your computer may take additional time for compilation. On a laptop with an RTX3060, `1jj2`, a pretty small system with 98k atoms, takes around 4 minutes. The list `pdbavail` should contain a list of pdb you want to compute their normal mode. The string `Benchmark_folder` should contain the path to store output. 

# Remark on JDM
Indeed, a simple rational filter `v = (A- \theta I )^-1 (-r)` will also work. See the discussion by Voss on Caley transform c.f. the original `Solving (I - uu^T )(A  - \theta I )(I - uu^T ) v  = -r`, though `(I - uu^T)` is 'satisfactorily' cheap. Also see the remark by Zhou in the Chebyshev Davidson paper, where they observed cases which `v = (A- \theta I )^-1 (-r)` is superior.

In [1]:
import glob
import platform

# A list of pdb available at different sizes

pdbavail = [ '../../DataRepo/PdbByAtomCount/1jj2.pdb' ] 
Benchmarking_folder = "../../VisualizationRepo/"


# No need to modify anything below, Press Run All

In [2]:

User_Platform = platform.system() # Windows Darwin Linux

User_rc_Gamma = 8.0
User_maxleafsize = 100
User_n_mode = 64
User_PlusI = 1.0
PDBCIF = "Pdb" if (".pdb" in pdbavail[0]) else "Cif"
User_MaxIter = 15000

# JDM Params

User_SolverName = 'gmres'
User_SolverMaxIter = 20
User_EigTolerance = 1e-12

User_IntegerOfIndexing = "INTEGER64"

PART00_Import = True
if PART00_Import:
   import os
   import gc
   import sys
   import pickle

   import numpy as np
   import time
   import tqdm

   import torch 


   import platform


   import time

   import cupy
   from cupy import cublas
   import cupy as cp
   import cupyx
   from cupyx.scipy.sparse import linalg as cupylinalg
   from cupyx.scipy import sparse as cupysparse




   sys.path.append('..')
   sys.path.append('../../')
   sys.path.append('../../InchingLiteInteger/Burn/')



   import InchingLiteInteger.util

   InchingLiteInteger.util.MkdirList([Benchmarking_folder])
   
   import InchingLiteInteger.Fuel.Coordinate.T1
   import InchingLiteInteger.Fuel.Coordinate.T2
   import InchingLiteInteger.Burn.Coordinate.T1
   import InchingLiteInteger.Burn.Coordinate.T3

   from InchingLiteInteger.Fuel.T1 import Xnumpy_SparseCupyMatrixUngappped

   import InchingLiteInteger.Burn.Visualisation.T1
   import InchingLiteInteger.Burn.Visualisation.T2


   from InchingLiteInteger.Burn.JacobiDavidsonHotellingDeflation.T1 import S_HeigvalJDMHD_HeigvecJDMHD
   from InchingLiteInteger.Burn.ThickRestartLanczosHotellingDeflation.T1 import S_HeigvalTRLMHD_HeigvecTRLMHD
   from InchingLiteInteger.Burn.ChebyshevDavidsonSubspaceIteration.T1 import S_HeigvalCDSIHD_HeigvecCDSIHD


   import InchingLiteInteger.Burn.HermitianLanczos.T2
   import InchingLiteInteger.Burn.PolynomialFilters.T0
   import InchingLiteInteger.Burn.PolynomialFilters.T2


   # ============================
   # Some torch speed up tips
   # =============================

   # Turn on cuda optimizer
   torch.backends.cudnn.is_available()
   torch.backends.cudnn.enabled = True
   torch.backends.cudnn.benchmark = True
   # disable debugs NOTE use only after debugging
   torch.autograd.set_detect_anomaly(False)
   torch.autograd.profiler.profile(False)
   torch.autograd.profiler.emit_nvtx(False)
   # Disable gradient tracking
   torch.no_grad()
   torch.inference_mode()
   torch.manual_seed(0)
   cupy.random.seed(seed = 0)
   os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # NOTE In case any error showup
   # Reset Cuda and Torch
   device = torch.device(0)
   torch.set_default_dtype(torch.float64)
   torch.set_default_tensor_type(torch.cuda.DoubleTensor)
   try:
      InchingLiteInteger.util.TorchEmptyCache()
   except RuntimeError:
      print("The GPU is free to use. THere is no existing occupant")
   try:
      print(torch.cuda.memory_summary(device = 0, abbreviated=True))
   except KeyError:
      print("The GPU is free to use. THere is no existing occupant")



/home/homingla/anaconda3/envs/V0/envs/Inching23/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The GPU is free to use. THere is no existing occupant
The GPU is free to use. THere is no existing occupant


In [3]:



for pdbfn in pdbavail:
    


    devices_ = [d for d in range(torch.cuda.device_count())]
    device_names_  = [torch.cuda.get_device_name(d) for d in devices_]
    User_Device =  device_names_[0]


    pdbid = pdbfn.split("/")[-1].split(".")[0]


    st = time.time()

    X_df, X_top = InchingLiteInteger.util.BasicPdbCifLoading(pdbfn)
    protein_xyz = X_df[['x','y','z']].to_numpy().astype(np.float64)
    # NOTE PDB format digit decimal do no destroy collinearity!
    n_atoms = protein_xyz.shape[0]



    # ===============================================
    # K-d Cuthill (NOTE CPU np array)
    # ===================================
    # NOTE Cuthill Order and Undo
    st = time.time()
    cuthill_order, cuthill_undoorder = InchingLiteInteger.Fuel.Coordinate.T1.X_KdCuthillMckeeOrder(protein_xyz,  
                                rc_Gamma = User_rc_Gamma, Reverse = True,
                                )
    protein_xyz = protein_xyz[cuthill_order,:]




    print('start eigsh cupy')

    mempool = cupy.get_default_memory_pool()
    pinned_mempool = cupy.get_default_pinned_memory_pool()

    # ==================
    # Cupy hessian
    # =====================
    PART03a_MakeCupyHessian = True
    if PART03a_MakeCupyHessian:
        # NOTE Nnz neighborhood after cuthill
        NnzMinMaxDict, HalfNnz  = InchingLiteInteger.Fuel.Coordinate.T1.X_KdUngappedMinMaxNeighbor(protein_xyz, 
                                    rc_Gamma = User_rc_Gamma, 
                                    maxleafsize = User_maxleafsize,
                                    CollectStat = False,
                                    User_ReturnHalfNnz = True,
                                    SliceForm= True)


        # NOTE Pyotch tensor spend textra memory when dlpack has to be called and there are mmeleak
        #X = torch.tensor(protein_xyz, device=device, requires_grad= False)
        X = protein_xyz

        Xnumpy_SparseCupyMatrixUngapppedC = Xnumpy_SparseCupyMatrixUngappped(X, batch_head = None,
            maxleafsize = User_maxleafsize, rc_Gamma = User_rc_Gamma,
            #device  = torch.device(0),
            User_PlusI = User_PlusI,
            #dtype_temp = torch.float64,
            #X_precision = torch.cuda.DoubleTensor,
            User_DictCharmmGuiPbc = None, #Dict_Pbc,
            NnzMinMaxDict = NnzMinMaxDict)
        if User_IntegerOfIndexing == "INTEGER32":
            A, A_diag = Xnumpy_SparseCupyMatrixUngapppedC.ReturnCupyHLowerTriangleInt32(
                            User_MaxHalfNnzBufferSize = HalfNnz)
        else:
            #print('gagag')
            A, A_diag = Xnumpy_SparseCupyMatrixUngapppedC.ReturnCupyHLowerTriangleInt64(
                            User_MaxHalfNnzBufferSize = HalfNnz)

        print("Matrix Index Datatype", A.indices.dtype)
        print("Matrix Datatype",A.data.shape)
        
        cupy.get_default_memory_pool().free_all_blocks()
        cupy.get_default_pinned_memory_pool().free_all_blocks()
        gc.collect()





    PART03b_MakeFreeModes = True
    if PART03b_MakeFreeModes:

        Q_HotellingDeflation = cp.zeros((6,3*n_atoms), dtype = cp.float64)
        # NOTE Translation
        for i in range(3):
            q1 = cp.zeros((n_atoms,3))
            q1[:,i] = 1/np.sqrt(n_atoms)
            Q_HotellingDeflation[i,:] = q1.flatten()
            q1 = None
            del q1
            cupy.get_default_memory_pool().free_all_blocks()
            cupy.get_default_pinned_memory_pool().free_all_blocks()



        # NOTE Rotation
        R_x = cp.array([        [0,0,0],
                                [0,0,-1],
                                [0,1,0]], dtype=cp.float64).T
        R_y = cp.array([        [0,0,1],
                                [0,0,0],
                                [-1,0,0]], dtype=cp.float64).T
        R_z = cp.array([        [0,-1,0],
                                [1,0,0],
                                [0,0,0]], dtype=cp.float64).T
        R_x = cupysparse.csr_matrix(R_x, dtype= cp.float64)
        R_y = cupysparse.csr_matrix(R_y, dtype= cp.float64)
        R_z = cupysparse.csr_matrix(R_z, dtype= cp.float64)
        gx = (cp.array(X)@R_x).flatten()
        Q_HotellingDeflation[3,:] = gx/ cp.linalg.norm(gx,ord=2)
        gy = (cp.array(X)@R_y).flatten()
        Q_HotellingDeflation[4,:] = gy/ cp.linalg.norm(gy,ord=2)
        gz = (cp.array(X)@R_z).flatten()
        Q_HotellingDeflation[5,:] = gz/ cp.linalg.norm(gz,ord=2)



        for i_FRO in range(2):
            V = Q_HotellingDeflation.T

            for ix in range(6):
                if ix == 0:
                    continue
                V[:,ix] -= cp.matmul(V[:,:ix], cp.matmul( V[:, :ix].T,V[:,ix] ))
                V[:,ix] /= cp.sqrt(V[:, ix].T @ V[:, ix]) # TODO torch.matmul or mvs
                V[:,ix] -= cp.matmul(V[:,:ix], cp.matmul( V[:, :ix].T,V[:,ix] ))
                V[:,ix] /= cp.sqrt(V[:, ix].T @ V[:, ix])
            Q_HotellingDeflation = V.T

        #gx = Q_HotellingDeflation[3]

        # NOTE Because of the naiive pbc implemented only 3 translational mode matters.
        Q_HotellingDeflation = cupyx.scipy.sparse.csr_matrix(Q_HotellingDeflation, dtype = cp.float64)

        gx, gy, gz = None, None, None
        del gx, gy, gz
        cupy.get_default_memory_pool().free_all_blocks()
        cupy.get_default_pinned_memory_pool().free_all_blocks()


    
    PART04_CalcualteEig = True
    if PART04_CalcualteEig:
        User_GapEstimate = None # NOTE Not in use.
        eigval, eigvec = S_HeigvalJDMHD_HeigvecJDMHD(A, A_diag,
                    k = User_n_mode,
                    tol = User_EigTolerance,
                    maxiter = User_MaxIter,
                    User_CorrectionSolverMaxiter = User_SolverMaxIter,
                    User_HalfMemMode= True,
                    User_IntermediateConvergenceTol=1e-3, # NOTE Do not touch for this problem
                    User_GapEstimate = User_GapEstimate, # NOTE This will be used for theta - gap_estimate
                    User_FactoringToleranceOnCorrection = 1e-4, # NOTE Do not touch for this problem
                    User_Q_HotellingDeflation= Q_HotellingDeflation,
                    User_HotellingShift = 40, # NOTE 40 is generally safe for first 64 modes, of course if you want to guarentee it you know a norm

                    )
        runtime = time.time() - st
        print("RUNNNTIME %s" %(runtime))
        peak_mem = cupy.get_default_memory_pool().used_bytes() / 1024 / 1024


        runtime = time.time() - st
        peak_mem = cupy.get_default_memory_pool().used_bytes() / 1024 / 1024
        with open("%s/Eigval_InchingJDM_%s_%s_%s.pkl" %(
                    Benchmarking_folder, pdbid, User_Platform, 
                    User_Device.replace(" ","")),"wb") as fn:
            pickle.dump(cupy.asnumpy(eigval) - User_PlusI ,fn, protocol=4)
        
        with open("%s/Eigvec_InchingJDM_%s_%s_%s.pkl" %(
                    Benchmarking_folder, pdbid, User_Platform, 
                    User_Device.replace(" ","")),"wb") as fn:    
            tempeigvec = cupy.asnumpy(eigvec)
            tempeigvec = tempeigvec.T
            tempeigvec = tempeigvec.reshape((int(User_n_mode),int(n_atoms),int(3)))
            pickle.dump(tempeigvec[:,cuthill_undoorder,:] ,fn, protocol=4)

        
        del tempeigvec
        gc.collect()
    





    PART05_Performance = True
    if PART05_Performance:
        #===================================
        # Check correct
        # =====================================

        User_HalfMemMode = True
        if User_HalfMemMode:
            KrylovAv = InchingLiteInteger.Burn.Krylov.T3.OOC2_HalfMemS_v_KrylovAv_VOID(A, A_diag)
        else:
            KrylovAv = InchingLiteInteger.Burn.Krylov.T3.OOC2_FullMemS_v_KrylovAv_VOID(A, A_diag)
        Av = cupy.empty((n_atoms*3,)).astype(A.dtype)


        delta_lambda_list = []
        for jj in range(User_n_mode):
            KrylovAv(A,cupy.ravel(eigvec[:,jj]),Av)
            B = Av - eigval[jj]* cupy.ravel(eigvec[:,jj])

            delta_lambda_list.append(cupy.asnumpy(cublas.nrm2(B)))
            #if jj < 20:
            print(eigval[jj], cupy.asnumpy(cublas.nrm2(B)))
        
        eigval = cupy.asnumpy(eigval)
        n_atoms = protein_xyz.shape[0]

        GPU = "%s %s" %(User_Platform, User_Device.replace(" GPU", ""))

        performance = ["Inching (JDM %s)" %(GPU), pdbfn, n_atoms, 
                        runtime, peak_mem, 
                        User_Platform, User_Device, 
                        User_maxleafsize]



        longperformance = []
        for i in range(len(delta_lambda_list)):
            longperformance.append(performance + [i ,delta_lambda_list[i], eigval[i] - User_PlusI])
        
        with open("%s/PerformanceList_InchingJDM_%s_%s_%s.pkl" %(Benchmarking_folder, 
            pdbid, User_Platform, User_Device.replace(" ","")),"wb") as fn:   
            pickle.dump(longperformance,fn, protocol=4)


        del X_df, protein_xyz
        gc.collect()



        B = None
        A.data = None
        A.indices = None
        A.indptr = None
        Q_HotellingDeflation = None
        del Q_HotellingDeflation

        del A.data, A.indices, A.indptr 
        del A, B
        Xnumpy_SparseCupyMatrixUngapppedC.X, Xnumpy_SparseCupyMatrixUngapppedC.X_unsqueezed = None, None
        del Xnumpy_SparseCupyMatrixUngapppedC.X, Xnumpy_SparseCupyMatrixUngapppedC.X_unsqueezed
        Xnumpy_SparseCupyMatrixUngapppedC = None
        del Xnumpy_SparseCupyMatrixUngapppedC
        eigvec, eigval = None, None
        del eigvec, eigval


        cupy.get_default_memory_pool().free_all_blocks()
        cupy.get_default_pinned_memory_pool().free_all_blocks()
        del X 
        torch.cuda.empty_cache()    
        torch.cuda.reset_peak_memory_stats(0)
        torch.cuda.memory_allocated(0)
        torch.cuda.max_memory_allocated(0)


100%|██████████| 99/99 [00:00<00:00, 170.04it/s]


N_neighbor within 8.0 angstrom Mean 99.97464051226368, Std 24.62011211610712
NN search in 0.5847101211547852 s


  0%|          | 0/98543 [00:01<?, ?it/s]


start eigsh cupy


100%|██████████| 1024/1024 [00:01<00:00, 841.91it/s]
/mnt/data0/homingla/Project-Inching/Inching-main/Notebook/Application20231128_DONE/../../InchingLiteInteger/Fuel/T1.py:218: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.frontal_gap_offset[i] = torch.tensor(


Mean number of Gaps > 100 is 17.998046875. Mean Gap Length Given Gap is 386.32246337493217
Max number of Gaps > 100 is 41. Max Gap Length Given Gap is 5318
Median number of Gaps > 100 is 19.0. Median Gap Length Given Gap is 224.0
Total Entry Savings 684354138 which is 66.2311271111961 percent of a Rectangular Batch
Nnz in Hessian (L+D) is 44776548.0. This will occupy 0.33030736818909645 GB for (L+D) data and at max 0.33030736818909645 GB for all indexings. Acceptable?


100%|██████████| 1024/1024 [00:04<00:00, 214.78it/s]


Matrix Index Datatype int64
Matrix Datatype (44473934,)
Start JDM Coarse Iter
0, 12.079113263483398, 33.9753282673743, 1e-06, 0
100, 0.000123689238630841, 1.0013109867764785, 1e-06, 0
200, 2.453303266448456e-11, 1.007502449083835, 1e-06, 3
300, 4.627123991473558e-07, 1.0246367240922498, 1e-06, 10
400, 5.265411003733234e-13, 1.0433909465903395, 1e-06, 17
400, 0.0037781366918755257, 1.0477358942135955, 1e-06, 18
500, 4.961205955639797e-07, 1.058356527869882, 1e-06, 26
600, 2.718582007423641e-13, 1.0720228377234622, 1e-06, 33
600, 0.005887347956501346, 1.0750355372145382, 1e-06, 34
700, 8.629725182048813e-09, 1.090970253130812, 1e-06, 42
800, 0.000225949435666824, 1.110922179066228, 1e-06, 50
900, 2.7447511688999206e-08, 1.1262197334896982, 1e-06, 58
DONE. We went through 977 coarse iter, 64 eigval converged
RUNNNTIME 69.89013338088989
1.000861819608412 5.947441236156044e-13
1.0013124193606726 8.959961946222426e-13
1.0056930046637207 7.708477175239602e-13
1.0075024490838351 8.845071693970